In [ ]:
import sys
import pprint
import requests
import ast
from collections import defaultdict
import pandas as pd
from pandas.io.json import json_normalize
import pickle
from functools import reduce
import pickle
import random

In [28]:
base_url = "https://developers.zomato.com/api/v2.1/"
api_keys = ["c1197244ad4605df38ec5e827e15a7b2",
            "86b123235ea063a6ac39687bbca355f0",
            "f819919aea69efed517042f48e1be4f2"]
NUM_CUISINES = 50

In [3]:
def initialize_app(config):
    return Zomato(config)


class Zomato:
    def __init__(self, config):
        self.user_key = config["user_key"]

    def get_categories(self):
        """
        Takes no input.
        Returns a dictionary of IDs and their respective category names.
        """
        headers = {'Accept': 'application/json', 'user-key': self.user_key}
        r = (requests.get(base_url + "categories", headers=headers).content).decode("utf-8")
        a = ast.literal_eval(r)

        self.is_key_invalid(a)
        self.is_rate_exceeded(a)

        categories = {}
        for category in a['categories']:
            categories.update({category['categories']['id'] : category['categories']['name']})

        return categories

    def get_city_ID(self, city_name):
        """
        Takes City Name as input.
        Returns the ID for the city given as input.
        """
        city_name = city_name.split(' ')
        city_name = '%20'.join(city_name)
        headers = {'Accept': 'application/json', 'user-key': self.user_key}
        r = (requests.get(base_url + "cities?q=" + city_name, headers=headers).content).decode("utf-8")
        a = ast.literal_eval(r)

        self.is_key_invalid(a)
        self.is_rate_exceeded(a)

        if len(a['location_suggestions']) == 0:
            raise Exception('invalid_city_name')
        elif 'name' in a['location_suggestions'][0]:
            city_name = city_name.replace('%20', ' ')
            if str(city_name).lower() in str(a['location_suggestions'][0]['name']).lower():
                return a['location_suggestions'][0]['id']
            else:
                raise ValueError('InvalidCityId')
                
    def get_city_name(self, city_ID):
        """
        Takes City ID as input.
        Returns the name of the city ID given as input.
        """
        self.is_valid_city_id(city_ID)

        headers = {'Accept': 'application/json', 'user-key': self.user_key}
        r = (requests.get(base_url + "cities?city_ids=" + str(city_ID), headers=headers).content).decode("utf-8")
        a = ast.literal_eval(r)

        self.is_key_invalid(a)
        self.is_rate_exceeded(a)

        if a['location_suggestions'][0]['country_name'] == "":
            raise ValueError('InvalidCityId')
        else:
            temp_city_ID = a['location_suggestions'][0]['id']
            if temp_city_ID == city_ID:
                return a['location_suggestions'][0]['name']

    def get_collections(self, city_ID, limit=None):
        """
        Takes City ID as input. limit parameter is optional.
        Returns dictionary of Zomato restaurant collections in a city and their respective URLs.
        """
        self.is_valid_city_id(city_ID)

        headers = {'Accept': 'application/json', 'user-key': self.user_key}
        if limit == None:
            r = (requests.get(base_url + "collections?city_id=" + str(city_ID), headers=headers).content).decode("utf-8")
        else:
            if str(limit).isalpha() == True:
                raise ValueError('LimitNotInteger')
            else:
                r = (requests.get(base_url + "collections?city_id=" + str(city_ID) + "&count=" + str(limit), headers=headers).content).decode("utf-8")
        a = ast.literal_eval(r)

        self.is_key_invalid(a)
        self.is_rate_exceeded(a)

        collections = {}
        for collection in a['collections']:
            collections.update({collection['collection']['title'] : collection['collection']['url']})

        return collections

    def get_cuisines(self, city_ID):
        """
        Takes City ID as input.
        Returns a sorted dictionary of all cuisine IDs and their respective cuisine names.
        """
        self.is_valid_city_id(city_ID)

        headers = {'Accept': 'application/json', 'user-key': self.user_key}
        r = (requests.get(base_url + "cuisines?city_id=" + str(city_ID), headers=headers).content).decode("utf-8")
        a = ast.literal_eval(r)

        self.is_key_invalid(a)
        self.is_rate_exceeded(a)

        if len(a['cuisines']) == 0:
            raise ValueError('InvalidCityId')
        temp_cuisines = {}
        cuisines = {}
        for cuisine in a['cuisines']:
            temp_cuisines.update({cuisine['cuisine']['cuisine_id'] : cuisine['cuisine']['cuisine_name']})

        for cuisine in sorted(temp_cuisines):
            cuisines.update({cuisine : temp_cuisines[cuisine]})

        return cuisines

    def get_establishment_types(self, city_ID):
        """
        Takes City ID as input.
        Returns a sorted dictionary of all establishment type IDs and their respective establishment type names.
        """
        self.is_valid_city_id(city_ID)

        headers = {'Accept': 'application/json', 'user-key': self.user_key}
        r = (requests.get(base_url + "establishments?city_id=" + str(city_ID), headers=headers).content).decode("utf-8")
        a = ast.literal_eval(r)

        self.is_key_invalid(a)
        self.is_rate_exceeded(a)

        temp_establishment_types = {}
        establishment_types = {}
        if 'establishments' in a:
            for establishment_type in a['establishments']:
                temp_establishment_types.update({establishment_type['establishment']['id'] : establishment_type['establishment']['name']})

            for establishment_type in sorted(temp_establishment_types):
                establishment_types.update({establishment_type : temp_establishment_types[establishment_type]})

            return establishment_types
        else:
            raise ValueError('InvalidCityId')

    def get_nearby_restaurants(self, latitude, longitude):
        """
        Takes the latitude and longitude as inputs.
        Returns a dictionary of Restaurant IDs and their corresponding Zomato URLs.
        """
        try:
            float(latitude)
            float(longitude)
        except ValueError:
            raise ValueError('InvalidLatitudeOrLongitude')

        headers = {'Accept': 'application/json', 'user-key': self.user_key}
        r = (requests.get(base_url + "geocode?lat=" + str(latitude) + "&lon=" + str(longitude), headers=headers).content).decode("utf-8")
        a = ast.literal_eval(r)

        nearby_restaurants = {}
        for nearby_restaurant in a['nearby_restaurants']:
            nearby_restaurants.update({nearby_restaurant['restaurant']['id'] : nearby_restaurant['restaurant']['url']})

        return nearby_restaurants

    def get_restaurant(self, restaurant_ID):
        """
        Takes Restaurant ID as input.
        Returns a dictionary of restaurant details.
        """
        self.is_valid_restaurant_id(restaurant_ID)

        headers = {'Accept': 'application/json', 'user-key': self.user_key}
        r = (requests.get(base_url + "restaurant?res_id=" + str(restaurant_ID), headers=headers).content).decode("utf-8")
        a = ast.literal_eval(r)

        if 'code' in a:
            if a['code'] == 404:
                raise('InvalidRestaurantId')

        restaurant_details = {}
        restaurant_details.update({"name" : a['name']})
        restaurant_details.update({"url" : a['url']})
        restaurant_details.update({"location" : a['location']['address']})
        restaurant_details.update({"city" : a['location']['city']})
        restaurant_details.update({"city_ID" : a['location']['city_id']})
        restaurant_details.update({"user_rating" : a['user_rating']['aggregate_rating']})

        restaurant_details = DotDict(restaurant_details)
        return restaurant_details

    def restaurant_search(self, entity_id="", entity_type="", latitude="", longitude="",
                          cuisines="",
                          start=0, limit=1000):
        """
        Takes either query, latitude and longitude or cuisine as input.
        Returns a list of Restaurant IDs.
        """
        cuisines = "%2C".join(cuisines.split(","))
        if str(limit).isalpha() == True:
            raise ValueError('LimitNotInteger')
        headers = {'Accept': 'application/json', 'user-key': self.user_key}
        r = requests.get(base_url + 
                         "search?entity_id=" + str(entity_id) +
                         "&entity_type=" + entity_type +
                         "&start=" + str(start) + 
                         "&count=" + str(limit) + 
                         "&lat=" + str(latitude) +
                         "&lon=" + str(longitude) + 
                         "&cuisines=" + str(cuisines), headers=headers)
        a = r.json()
        try:
            if a['results_found'] == 0:
                return []
            else:
                return a['restaurants']
        except:
            print(a)

    def is_valid_restaurant_id(self, restaurant_ID):
        """
        Checks if the Restaurant ID is valid or invalid.
        If invalid, throws a InvalidRestaurantId Exception.
        """
        restaurant_ID = str(restaurant_ID)
        if restaurant_ID.isnumeric() == False:
            raise ValueError('InvalidRestaurantId')

    def is_valid_city_id(self, city_ID):
        """
        Checks if the City ID is valid or invalid.
        If invalid, throws a InvalidCityId Exception.
        """
        city_ID = str(city_ID)
        if city_ID.isnumeric() == False:
            raise ValueError('InvalidCityId')

    def is_key_invalid(self, a):
        """
        Checks if the API key provided is valid or invalid.
        If invalid, throws a InvalidKey Exception.
        """
        if 'code' in a:
            if a['code'] == 403:
                raise ValueError('InvalidKey')



    def is_rate_exceeded(self, a):
        """
        Checks if the request limit for the API key is exceeded or not.
        If exceeded, throws a ApiLimitExceeded Exception.
        """
        if 'code' in a:
            if a['code'] == 440:
                raise Exception('ApiLimitExceeded')

In [ ]:
pp = pprint.PrettyPrinter(indent=4)
config={"user_key":api_keys[0]}
zomato = initialize_app(config)

## Cities

In [9]:
city_names = ['Las Vegas', 'Phoenix', 'Toronto', 'Charlotte', 'Scotsdale', 'Calgary',
              'Pittsburg', 'Mesa', 'Montreal', 'Henderson', 'Cleveland', 'Mesa', 'Madidson']

In [11]:
city2id = {}
for city_name in city_names:
    try:
        city2id[city_name] = zomato.get_city_ID(city_name)
    except Exception as e:
        print(e)
        print(city_name)
print()
print(city2id)

invalid_city_name
Madidson


## Cuisines

In [24]:
id2cuisine = zomato.get_cuisines(city2id['Las Vegas'])
cuisine2id = {v: k for k, v in cuisine2id.items()}
len(cuisine2id)

99

In [29]:
assert len(city2id) * 5 * NUM_CUISINES < len(api_keys) * 999
cuisine_ids = random.sample(list(cuisine2id.values()), num_cuisines)
fast_food_id = cuisine2id['Fast Food']
if fast_food_id not in cuisine_ids:
    cuisine_ids.pop()
    cuisine_ids += [fast_food_id]
print(len(cuisine_ids), fast_food_id in cuisine_ids)

50 True


## Restaurants

In [38]:
config={"user_key": api_keys[2]}
zomato = initialize_app(config)

In [ ]:
city2res = defaultdict(list)

In [37]:
num_res = 0
api_idx = 1
config={"user_key": api_keys[api_idx]}
zomato = initialize_app(config)

for city_name, city_id in city2id.items():
    if city_name in city2res:
        continue
        
    for cuisine_id in cuisine_ids:
        for start in range(0, 100, 20):
            try:
                city2res[city_name].extend(zomato.restaurant_search(entity_id=city_id,
                                                                    entity_type='city',
                                                                    start=start,
                                                                    cuisines=str(cuisine_id),
                                                                    limit=1000))
                print(city_name, len(city2res[city_name]))
                num_res += 20
            except Exception as e:
                print(e, num_res)
                api_idx += 1
                if api_idx >= len(api_keys):
                    print(abc)
                config={"user_key": api_keys[api_idx]}
                zomato = initialize_app(config)

Scotsdale 3
Scotsdale 3
Scotsdale 3
Scotsdale 3
{'code': 440, 'status': '', 'message': 'API limit exceeded'}
'NoneType' object is not iterable 80
{'code': 440, 'status': '', 'message': 'API limit exceeded'}
'NoneType' object is not iterable 80


NameError: name 'abc' is not defined

In [40]:
for k, v in city2res.items():
    print(k, len(v))

Las Vegas 2151
Phoenix 1578
Toronto 3333
Charlotte 1622
Scotsdale 3


In [44]:
full_json_data = []
for city, restaurants in city2res.items():
    for restaurant in restaurants:
        record = restaurant['restaurant'].copy()
        record.pop('R', None)
        record.pop('featured_image', None)
        record.pop('thumb', None)
        record.pop('apikey', None)
        record.pop('deeplink', None)
        keys = list(record.keys()).copy()
        for k in keys:
            if 'url' in k:
                record.pop(k, None)

        normal_record = json_normalize(record)
        full_json_data.append(normal_record)

In [45]:
df = pd.concat(full_json_data)

In [46]:
for col in df.columns:
    print(col)

average_cost_for_two
cuisines
currency
establishment_types
has_online_delivery
has_table_booking
id
include_bogo_offers
is_book_form_web_view
is_delivering_now
is_table_reservation_supported
is_zomato_book_res
location.address
location.city
location.city_id
location.country_id
location.latitude
location.locality
location.locality_verbose
location.longitude
location.zipcode
medio_provider
mezzo_provider
name
offers
opentable_support
price_range
switch_to_order_menu
user_rating.aggregate_rating
user_rating.custom_rating_text
user_rating.custom_rating_text_background
user_rating.rating_color
user_rating.rating_text
user_rating.rating_tool_tip
user_rating.votes


In [47]:
df.applymap(lambda x: x == '').sum()

average_cost_for_two                            0
cuisines                                        0
currency                                        0
establishment_types                             0
has_online_delivery                             0
has_table_booking                               0
id                                              0
include_bogo_offers                             0
is_book_form_web_view                           0
is_delivering_now                               0
is_table_reservation_supported                  0
is_zomato_book_res                              0
location.address                                0
location.city                                   0
location.city_id                                0
location.country_id                             0
location.latitude                               0
location.locality                               0
location.locality_verbose                       0
location.longitude                              0


In [48]:
df.isnull().sum()

average_cost_for_two                            0
cuisines                                        0
currency                                        0
establishment_types                             0
has_online_delivery                             0
has_table_booking                               0
id                                              0
include_bogo_offers                             0
is_book_form_web_view                           0
is_delivering_now                               0
is_table_reservation_supported                  0
is_zomato_book_res                              0
location.address                                0
location.city                                   0
location.city_id                                0
location.country_id                             0
location.latitude                               0
location.locality                               0
location.locality_verbose                       0
location.longitude                              0


In [49]:
df['location.city'].value_counts()

Toronto                3330
Las Vegas              2147
Phoenix                1577
Charlotte              1542
Rock Hill                75
Boulder City              4
China Grove               3
Niagara-on-the-Lake       3
Mt Pleasant               2
Denmark                   2
Rest of Arizona           1
Walpole                   1
Name: location.city, dtype: int64

In [50]:
g = df.columns.to_series().groupby(df.dtypes).groups

In [56]:
df['average_cost_for_two'].value_counts()

10          2044
25          1690
40           770
20           604
15           566
30           537
70           338
0            334
35           328
50           252
13           229
45           216
60           164
65           138
80           107
100           88
43            77
55            52
75            36
90            26
85            21
150           18
120           16
110            9
140            6
95             4
125            3
160            3
5              2
145            2
180            2
25000017       1
300            1
27             1
25050          1
130            1
Name: average_cost_for_two, dtype: int64

In [57]:
df.describe()

,average_cost_for_two,has_online_delivery,has_table_booking,is_book_form_web_view,is_delivering_now,is_table_reservation_supported,is_zomato_book_res,location.city_id,location.country_id,opentable_support,price_range,switch_to_order_menu
count,8.687000e+03,8687.0,8687.000000,8687.0,8687.0,8687.000000,8687.000000,8687.000000,8687.000000,8687.0,8687.000000,8687.0
mean,2.909364e+03,0.0,0.001496,0.0,0.0,0.001496,0.001496,231.448026,147.252101,0.0,1.814320,0.0
std,2.682286e+05,0.0,0.038658,0.0,0.0,0.038658,0.038658,273.113811,87.075126,0.0,0.911809,0.0
min,0.000000e+00,0.0,0.000000,0.0,0.0,0.000000,0.000000,89.000000,14.000000,0.0,1.000000,0.0
25%,1.000000e+01,0.0,0.000000,0.0,0.0,0.000000,0.000000,89.000000,37.000000,0.0,1.000000,0.0
50%,2.500000e+01,0.0,0.000000,0.0,0.0,0.000000,0.000000,282.000000,216.000000,0.0,2.000000,0.0
75%,4.000000e+01,0.0,0.000000,0.0,0.0,0.000000,0.000000,301.000000,216.000000,0.0,2.000000,0.0
max,2.500002e+07,0.0,1.000000,0.0,0.0,1.000000,1.000000,7625.000000,216.000000,0.0,4.000000,0.0


In [ ]:
df['is_delivering_now'].

In [54]:
df['has_table_booking'].value_counts()

0    8674
1      13
Name: has_table_booking, dtype: int64

In [61]:
g

{dtype('bool'): Index(['include_bogo_offers'], dtype='object'),
 dtype('int64'): Index(['average_cost_for_two', 'has_online_delivery', 'has_table_booking',
        'is_book_form_web_view', 'is_delivering_now',
        'is_table_reservation_supported', 'is_zomato_book_res',
        'location.city_id', 'location.country_id', 'opentable_support',
        'price_range', 'switch_to_order_menu'],
       dtype='object'),
 dtype('O'): Index(['cuisines', 'currency', 'establishment_types', 'id', 'location.address',
        'location.city', 'location.latitude', 'location.locality',
        'location.locality_verbose', 'location.longitude', 'location.zipcode',
        'medio_provider', 'mezzo_provider', 'name', 'offers',
        'user_rating.aggregate_rating', 'user_rating.custom_rating_text',
        'user_rating.custom_rating_text_background', 'user_rating.rating_color',
        'user_rating.rating_text', 'user_rating.rating_tool_tip',
        'user_rating.votes'],
       dtype='object')}

In [79]:
for col in df.columns:
    print(col.upper())
    try:
        print(f"{df[col].value_counts()[:5]}")
    except:
        continue
    print('\n\n')

AVERAGE_COST_FOR_TWO
10    2044
25    1690
40     770
20     604
15     566
Name: average_cost_for_two, dtype: int64



CUISINES
Mexican      319
Fast Food    220
Desserts     198
Seafood      136
Italian      134
Name: cuisines, dtype: int64



CURRENCY
$    8687
Name: currency, dtype: int64



ESTABLISHMENT_TYPES
HAS_ONLINE_DELIVERY
0    8687
Name: has_online_delivery, dtype: int64



HAS_TABLE_BOOKING
0    8674
1      13
Name: has_table_booking, dtype: int64



ID
18290103    5
17031909    5
18251866    5
18677591    4
17031244    4
Name: id, dtype: int64



INCLUDE_BOGO_OFFERS
True    8687
Name: include_bogo_offers, dtype: int64



IS_BOOK_FORM_WEB_VIEW
0    8687
Name: is_book_form_web_view, dtype: int64



IS_DELIVERING_NOW
0    8687
Name: is_delivering_now, dtype: int64



IS_TABLE_RESERVATION_SUPPORTED
0    8674
1      13
Name: is_table_reservation_supported, dtype: int64



IS_ZOMATO_BOOK_RES
0    8674
1      13
Name: is_zomato_book_res, dtype: int64



LOCATION.ADDRESS
Phoenix

In [76]:
for col in df.columns:
    print(col)
    print(df[col].value_counts())

average_cost_for_two
10          2044
25          1690
40           770
20           604
15           566
30           537
70           338
0            334
35           328
50           252
13           229
45           216
60           164
65           138
80           107
100           88
43            77
55            52
75            36
90            26
85            21
150           18
120           16
110            9
140            6
95             4
125            3
160            3
5              2
145            2
180            2
25000017       1
300            1
27             1
25050          1
130            1
Name: average_cost_for_two, dtype: int64
cuisines
Mexican                                       319
Fast Food                                     220
Desserts                                      198
Seafood                                       136
Italian                                       134
American, Breakfast                           116
Coffee and Tea, T

TypeError: unhashable type: 'list'

In [65]:
df['curren'].value_counts()

0    8674
1      13
Name: has_table_booking, dtype: int64

In [63]:
df['average_cost_for_two'].value_counts()

10          2044
25          1690
40           770
20           604
15           566
30           537
70           338
0            334
35           328
50           252
13           229
45           216
60           164
65           138
80           107
100           88
43            77
55            52
75            36
90            26
85            21
150           18
120           16
110            9
140            6
95             4
125            3
160            3
5              2
145            2
180            2
25000017       1
300            1
27             1
25050          1
130            1
Name: average_cost_for_two, dtype: int64

In [62]:
df['user_rating.rating_text']

0    Excellent
0    Very Good
0    Very Good
0    Excellent
0         Good
0         Good
0         Good
0    Very Good
0    Very Good
0         Good
0         Good
0         Good
0         Good
0    Very Good
0         Good
0         Good
0         Good
0         Good
0         Good
0         Good
0         Good
0      Average
0      Average
0         Good
0         Good
0         Good
0         Good
0         Good
0         Good
0         Good
       ...    
0    Not rated
0    Not rated
0    Not rated
0    Not rated
0    Not rated
0    Not rated
0    Not rated
0    Not rated
0    Not rated
0    Not rated
0    Not rated
0    Not rated
0    Not rated
0         Good
0         Good
0         Good
0         Good
0         Good
0      Average
0      Average
0      Average
0      Average
0      Average
0      Average
0      Average
0      Average
0    Not rated
0    Very Good
0         Good
0    Not rated
Name: user_rating.rating_text, Length: 8687, dtype: object

In [ ]:
df.to_csv()